* you need to make model pipline and testing the model with it (error)

* Saving confusion matrix in MLFLOW (Good)
 
* Need to add sqlite backend and automatic regristary the model
  
* add experiment name and add vresion

* add automatic select the best model and predict the model with it with new unseen dataset.
  
* Train test val spliiting in dataset
  
* Make result dataframe and output file

* MOve all parameters to main file and remove from others

* make argv to make sure what model need to be train. 

* print to logger
  
* Make new dataset for testing deploy model

* Deployment to flask and docker
* 

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.set_experiment("my-experiment-1")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1682395858287, experiment_id='1', last_update_time=1682395858287, lifecycle_stage='active', name='my-experiment-1', tags={}>

In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1682395858287, experiment_id='1', last_update_time=1682395858287, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1682395714480, experiment_id='0', last_update_time=1682395714480, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
mlflow.get_experiment(1)

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1682395858287, experiment_id='1', last_update_time=1682395858287, lifecycle_stage='active', name='my-experiment-1', tags={}>

In [6]:
import pandas as pd
import numpy as np
# Evaluation metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score,confusion_matrix,classification_report
# MLflow
import mlflow
import mlflow.sklearn
# others
import json
import yaml
import joblib
import argparse
from urllib.parse import urlparse
import logging
import warnings
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

import sys
# caution: path[0] is reserved for script path (or '' in REPL)

# from params_loader import read_params
# from models import build_and_load_models
# from check_data_exist import check_dataset_exist
import warnings
warnings.filterwarnings("ignore")
# np.random.seed(40)


In [7]:

# SKlearn model
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import  RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import NuSVC
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from catboost import Pool, CatBoostClassifier, cv
# import lightgbm as lgb
import xgboost as xgb


def build_and_load_models():
    # Models
    Models = {
        "Logistic Regression": LogisticRegression(),                    #
        "Support Vector Classifier": SVC(),                             # Ridge, SVC, LinearSVC, Passive_AC
        "Decision Tree": DecisionTreeClassifier(max_depth=6),           #
        "KNearest": KNeighborsClassifier(n_neighbors=5),                # doesn't have model.predict_proba so I left out.
        "GaussianNB" : GaussianNB(),                                    #
        "LDA" : LinearDiscriminantAnalysis(),                           # 
        "Ridge" : RidgeClassifier(),                                    #  
        "QDA" : QuadraticDiscriminantAnalysis(),                        #
        "Bagging" : BaggingClassifier(),                                #
        "MLP" : MLPClassifier(),                                        #
        "LSVC" : LinearSVC(),                                           #  
        "BernoulliNB" : BernoulliNB(),                                  #  
        "Passive_AC" : PassiveAggressiveClassifier(),                   # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #  
        "SGB"     : GradientBoostingClassifier(n_estimators=100, random_state=9),
        "Adaboost" : AdaBoostClassifier(n_estimators=100, random_state=9, algorithm='SAMME.R', learning_rate=0.8),
        "Extra_T" : ExtraTreesClassifier(n_estimators=100, max_features=3),
        "R_forest" : RandomForestClassifier(max_samples=0.9, n_estimators=100, max_features=3),
        # "XGB" : xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)
        }
    return Models
 

### Add lgb, catboost and Stacking and ANN


### virtualize end result



In [8]:
train_data_path = '../../data/processed/undersampling_test.parquet'
test_data_path  = '../../data/processed/undersampling_test.parquet'
target = 'Class'

In [9]:
sys.path.append("../data/")
from data_function import get_feat_and_target
def features_labels_split(train_data_path, test_data_path, target):
    train_dataset = pd.read_parquet(train_data_path)
    test_dataset = pd.read_parquet(test_data_path)
    train_features,train_label = get_feat_and_target(train_dataset, target)
    test_features,test_label = get_feat_and_target(test_dataset, target)
    return train_features, train_label, test_features, test_label

In [10]:
train_dataset = pd.read_parquet(train_data_path)
test_dataset = pd.read_parquet(test_data_path)
train_features,train_label = get_feat_and_target(train_dataset, target)
test_features,test_label = get_feat_and_target(test_dataset, target)

spliiting X and y
spliiting X and y


In [11]:
 Models = build_and_load_models()

In [12]:
Models

{'Logistic Regression': LogisticRegression(),
 'Support Vector Classifier': SVC(),
 'Decision Tree': DecisionTreeClassifier(max_depth=6),
 'KNearest': KNeighborsClassifier(),
 'GaussianNB': GaussianNB(),
 'LDA': LinearDiscriminantAnalysis(),
 'Ridge': RidgeClassifier(),
 'QDA': QuadraticDiscriminantAnalysis(),
 'Bagging': BaggingClassifier(),
 'MLP': MLPClassifier(),
 'LSVC': LinearSVC(),
 'BernoulliNB': BernoulliNB(),
 'Passive_AC': PassiveAggressiveClassifier(),
 'SGB': GradientBoostingClassifier(random_state=9),
 'Adaboost': AdaBoostClassifier(learning_rate=0.8, n_estimators=100, random_state=9),
 'Extra_T': ExtraTreesClassifier(max_features=3),
 'R_forest': RandomForestClassifier(max_features=3, max_samples=0.9)}

In [13]:
def eval_metrics(classifier, test_features, test_labels, avg_method):
    
    # make prediction
    predictions   = classifier.predict(test_features)
    base_score   = classifier.score(test_features,test_labels)
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions, average=avg_method)
    recall = recall_score(test_labels, predictions, average=avg_method)
    f1score = f1_score(test_labels, predictions, average=avg_method)
    Matrix = confusion_matrix(test_labels, predictions)
    matrix_scores = { 
        "true negative"  : Matrix[0][0],
        "false positive" : Matrix[0][1],
        "false negative" : Matrix[1][0],
        "true positive " : Matrix[1][1]
    }
    target_names = ['0','1']
    print("Classification report")
    print("---------------------","\n")
    print(classification_report(test_labels, predictions,target_names=target_names),"\n")
    print("Confusion Matrix")
    print("---------------------","\n")
    print(f"{confusion_matrix} \n")

    print("Accuracy Measures")
    print("---------------------","\n")
    print("Base score: ", base_score)
    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1score)
    
    return base_score,accuracy,precision,recall,f1score,matrix_scores

In [14]:

i = 1
for Model_Name, classifier in Models.items():   
        print(f"{i}. {Model_Name}")
        # fit the model
        classifier.fit(train_features, train_label)
        i = i+1
        with mlflow.start_run():
        # Calculate the metrics
                base_score,accuracy,precision,recall,f1score,matrix_scores = eval_metrics(classifier, 
                                                                        test_features, 
                                                                        test_label, 
                                                                        'weighted')
                
                mlflow.log_param("Models"            , Model_Name)
                mlflow.log_params(matrix_scores)
                mlflow.log_metric("base_score"      , base_score)
                mlflow.log_metric("accuary"         , accuracy)
                mlflow.log_metric("av_precision"    , precision)
                mlflow.log_metric("recall"          , recall)
                mlflow.log_metric("f1"              , f1score)

1. Logistic Regression
